In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import time
import matplotlib.pyplot as plt
import math
import copy
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from tensorflow.keras.layers import Input
from sklearn.preprocessing import StandardScaler
from keras.layers import Dropout

In [2]:
# HIPERPARÂMETROS:
start_date = '2014-08-21'
W = 250  # Lembrando que o tamanho do treinamento será W - w
w = 14

In [3]:
# Criptomoedas que vamos escolher para nosso portifólio
cryptos = [
    'BTC-USD', 'LTC-USD'
]

In [4]:
## Função para calcular o retorno logarítmico
def log_return(series):
    return np.log(series['Close'] / series['Open'])

In [5]:
data = {}
for crypto in cryptos:
    # Baixar dados e calcular retorno logarítmico
    df = yf.download(crypto, start=start_date, end=pd.to_datetime("today").strftime("%Y-%m-%d"), interval='1d')
    df['Return'] = log_return(df)

    # Construir DataFrame final, pegando o retorno do dia anterior e o atual
    df_final = pd.DataFrame({
        #'Volume': df['Volume'],
        'Crypto_Return_Today': df['Return']
    }).dropna()

    data[crypto] = df_final

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [6]:
data['LTC-USD']

,Crypto_Return_Today
Date,
2014-09-17 00:00:00+00:00,-0.005390
2014-09-18 00:00:00+00:00,-0.078024
2014-09-19 00:00:00+00:00,-0.079802
2014-09-20 00:00:00+00:00,-0.009926
2014-09-21 00:00:00+00:00,-0.004031
...,...
2024-11-07 00:00:00+00:00,0.006597
2024-11-08 00:00:00+00:00,0.018171
2024-11-09 00:00:00+00:00,0.017163


In [7]:
data_complete = {}
for crypto in cryptos:
    # Baixar dados e calcular retorno logarítmico
    df = yf.download(crypto, start=start_date, end=pd.to_datetime("today").strftime("%Y-%m-%d"), interval='1d')
    df['Return'] = log_return(df)

    # Construir DataFrame final, pegando o retorno do dia anterior e o atual
    df_final = pd.DataFrame({
        #'Volume': df['Volume'],
        'Crypto_Return_Today': df['Return']
    }).dropna()

    # Adicionar a coluna 'exp_value' com valores NaN
    df_final['exp_value'] = np.nan

    # Armazenar no dicionário
    data_complete[crypto] = df_final

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:
print(len(data_complete['LTC-USD']))
data_complete['LTC-USD']

3709


,Crypto_Return_Today,exp_value
Date,,
2014-09-17 00:00:00+00:00,-0.005390,NaN
2014-09-18 00:00:00+00:00,-0.078024,NaN
2014-09-19 00:00:00+00:00,-0.079802,NaN
2014-09-20 00:00:00+00:00,-0.009926,NaN
2014-09-21 00:00:00+00:00,-0.004031,NaN
...,...,...
2024-11-07 00:00:00+00:00,0.006597,NaN
2024-11-08 00:00:00+00:00,0.018171,NaN
2024-11-09 00:00:00+00:00,0.017163,NaN


In [9]:
# Criando a função First Transformation, que recebe uma tabela contendo os retornos de todos os dias dos ultimos X anos e retorno um vetor, que cada elemento
# é uma tabela do retorno de W+1 dias.
def FirstTransform(df, W):
    vet = []
    Linhas, Colunas = df.shape
    for i in range(Linhas, W+1, -1):
        vet.append(df.iloc[(i-W-1):i])
    return vet

In [10]:
# Aqui eu perco W+1 dias do meu dataset
for crypto in cryptos:
    data[crypto] = FirstTransform(data[crypto], W)

In [11]:
print(len(data['LTC-USD']))
data['LTC-USD'][0]

3458


,Crypto_Return_Today
Date,
2024-03-06 00:00:00+00:00,0.045173
2024-03-07 00:00:00+00:00,0.026196
2024-03-08 00:00:00+00:00,0.005920
2024-03-09 00:00:00+00:00,0.024694
2024-03-10 00:00:00+00:00,-0.037443
...,...
2024-11-07 00:00:00+00:00,0.006597
2024-11-08 00:00:00+00:00,0.018171
2024-11-09 00:00:00+00:00,0.017163


## 2.2 Separar em treinamento e teste - Permitir Bagging

In [12]:
test_target = {}
for crypto in cryptos:
    row = []
    for i in range(len(data[crypto])):
        row.append((pd.DataFrame(data[crypto][i].iloc[-1])).T)
        data[crypto][i] = data[crypto][i].drop(data[crypto][i].index[-1])
    test_target[crypto] = row

In [13]:
data['LTC-USD'][0], test_target['LTC-USD'][0]['Crypto_Return_Today']

(                           Crypto_Return_Today
 Date                                          
 2024-03-06 00:00:00+00:00             0.045173
 2024-03-07 00:00:00+00:00             0.026196
 2024-03-08 00:00:00+00:00             0.005920
 2024-03-09 00:00:00+00:00             0.024694
 2024-03-10 00:00:00+00:00            -0.037443
 ...                                        ...
 2024-11-06 00:00:00+00:00             0.081865
 2024-11-07 00:00:00+00:00             0.006597
 2024-11-08 00:00:00+00:00             0.018171
 2024-11-09 00:00:00+00:00             0.017163
 2024-11-10 00:00:00+00:00             0.032670
 
 [250 rows x 1 columns],
 2024-11-11 00:00:00+00:00    0.045925
 Name: Crypto_Return_Today, dtype: float64)

## 2.3 Bagging

In [14]:
# Definindo uma função bagging, que recebe um dataframe e retorna um vetor de dataframes.
def Bagging(df, n, gamma):
    df_bagged = []
    for i in range(gamma):
        aux = df.sample(n = n, random_state = i)
        aux = aux.sort_index()
        df_bagged.append(aux)
    return df_bagged

# 3. Machine Learning

* vamos peimeiro fazer do modo 1 para todos os dias
* depois aplicar para os ultimos 290 dias o modo 2 (contém bagging e purge K-Fold-CV)

## 3.1 Função do Modelo


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau


def create_model(a, b):
    model = Sequential([
        Input(shape=(a, b)),
        LSTM(units=64, return_sequences=True), #, kernel_regularizer=l2(0.002)),
        Dropout(0.5),
        BatchNormalization(),
        LSTM(units=32, return_sequences=False),
        Dropout(0.5),
        Dense(units=1)
    ])
    optimizer = RMSprop(learning_rate=1e-5)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model


In [16]:
# Função do modelo LSTM
def Model(df, time_step, features, model=None):
    # Normalizando o dataframe
    #scaler = StandardScaler()
    #df_scaled = scaler.fit_transform(df)
    #df_array = np.array(df_scaled)
    df_array = np.array(df)

    # Transformando no formato LSTM
    X_lstm = []
    y_lstm = []
    for i in range(time_step, len(df_array)):
        X_lstm.append(df_array[i-time_step:i, :])  # Pega 'timesteps' linhas anteriores
        y_lstm.append(df_array[i])  # Alvo é o valor do dia seguinte
    X_lstm = np.array(X_lstm)
    y_lstm = np.array(y_lstm)

    # Conjunto de validação - primeiros 30 elementos
    X_val = X_lstm[-20:-1]
    y_val = y_lstm[-20:-1]
    y_val = pd.DataFrame(y_val)[features - 1].to_numpy().reshape((len(y_val), 1))

    # Separar o X_train (todos exceto o último elemento)
    X_train = X_lstm[:-20]
    y_train = y_lstm[:-20]
    y_train = pd.DataFrame(y_train)[features - 1].to_numpy()
    y_train = y_train.reshape((len(y_train), 1))
    y_train.shape

    # X_test e y_test
    X_test = X_lstm[-1:]
    y_test = y_lstm[-1:]
    y_test = y_lstm[-1:]
    y_test = pd.DataFrame(y_test)[features - 1].to_numpy()
    y_test = y_test.reshape((len(y_test), 1))

    # Treinar o modelo específico para a criptomoeda

    early_stopping = EarlyStopping(monitor='val_loss', patience=40, restore_best_weights=True)
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs= 300, verbose=0, callbacks=[early_stopping])
    # Fazer a previsão
    y_hat_scaled = model.predict(X_test)


    return y_hat_scaled

##  3.2 Treinamento com Paralelismo e Geração dos Expected Values

In [17]:
def Add(y_hat, data_complete, ind, crypto):
    data_complete[crypto].at[ind, 'exp_value'] = y_hat

In [18]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Usando GPU:", physical_devices[0])
else:
    print("Usando CPU")

Usando GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [19]:
for crypto in cryptos:
    print(len(data[crypto]))
len(data['LTC-USD'][0])

3458
3458


250

In [20]:
test_target['LTC-USD'][2550]
test_target['LTC-USD'][2550]

,Crypto_Return_Today
2017-11-18 00:00:00+00:00,0.03267


In [21]:
aux = data_complete['LTC-USD'].loc['2017-11-08':'2018-09-04']
num_positive = (aux['Crypto_Return_Today'] > 0).sum()
num_negative = (aux['Crypto_Return_Today'] < 0).sum()

print("\n\n--------------------------------------------------------")
print("\n\nCriptomoeda: ", 'LTC')
print("\n                              Real")
print("\nNumero de Subidas: ",num_positive,"   |   ",100*num_positive / (num_negative + num_positive),"%")
print("\nNumero de Quedas : ",num_negative,"   |   ",100*num_negative / (num_negative + num_positive),"%")



--------------------------------------------------------


Criptomoeda:  LTC

                              Real

Numero de Subidas:  140    |    46.51162790697674 %

Numero de Quedas :  161    |    53.48837209302326 %


In [ ]:
# Função que processa cada criptomoeda (paralelismo será aplicado aqui)
def process_crypto(crypto, data, test_target, data_complete, time_step, features):
    # Criar um modelo para cada criptomoeda
    for day in range(2550, 2718):
        model = create_model(time_step, features)

        df_train = data[crypto][day].copy()  # df é uma tabela de 300 linhas e uma coluna
        df_test = test_target[crypto][day].copy()  # df_test é uma tabela com 1 linha e uma coluna
        df = pd.concat([df_train, df_test], ignore_index=True)  # Unindo as duas tabelas

        scaler = StandardScaler()
        df = scaler.fit_transform(df)

        y_hat_scaled = Model(df, time_step, features, model=model)  # Chamar o modelo para previsão

        y_hat_scaled = np.repeat(y_hat_scaled, features, axis=1)
        y_hat_scaled = pd.DataFrame(y_hat_scaled, columns=(['Predictions_1']))

        y_hat = scaler.inverse_transform(y_hat_scaled)[0][features - 1]

        Add(y_hat, data_complete, df_test.index[0], crypto)  # Adicionar o resultado na tabela final

    print("\nMoeda: ", crypto, " processada")

# Número de samples que vamos testar o modelo
time_step = w
features = 1

# Criar o executor para rodar as criptomoedas em paralelo
with ThreadPoolExecutor() as executor:
    # Executar o processo para cada criptomoeda simultaneamente
    futures = [
        executor.submit(process_crypto, crypto, data, test_target, data_complete, time_step, features)
        for crypto in cryptos
    ]

    # Aguardar a conclusão de todas as threads
    for future in futures:
        future.result()

print("Processamento em paralelo finalizado.")

42min para 100 dias


5e-3 sem kernel paciencia 70
Pre_media  0.782608695652174
Acc_media  0.6
MAE_media  0.04457564047387402

1e-3 paciencia 70 sem kernel


In [23]:
df = data_complete['BTC-USD']
df.dropna()


,Crypto_Return_Today,exp_value
Date,,
2018-04-18 00:00:00+00:00,0.027192,0.009423
2018-04-19 00:00:00+00:00,0.016415,0.003857
2018-04-20 00:00:00+00:00,0.065273,0.005372
2018-04-21 00:00:00+00:00,0.005274,0.005664
2018-04-22 00:00:00+00:00,-0.013832,0.001993
...,...,...
2018-10-30 00:00:00+00:00,-0.000437,-0.002050
2018-10-31 00:00:00+00:00,-0.003063,-0.001979
2018-11-01 00:00:00+00:00,0.009395,-0.001207


In [24]:
data_visual = data_complete

In [25]:
results = []

for crypto in cryptos:
    df = data_visual[crypto]
    df['ae'] = df['exp_value'] - df['Crypto_Return_Today']
    df['ae'] = abs(df['ae'])
    df = df.dropna()

    # Cálculos dos valores
    num_positive = (df['Crypto_Return_Today'] > 0).sum()
    num_negative = (df['Crypto_Return_Today'] < 0).sum()
    hat_pos = (df['exp_value'] > 0).sum()
    hat_neg = (df['exp_value'] < 0).sum()

    # Calcula a porcentagem de subidas e quedas reais e previstas
    total_real = num_positive + num_negative
    total_pred = hat_pos + hat_neg
    perc_positive_real = (100 * num_positive / total_real) if total_real > 0 else None
    perc_positive_pred = (100 * hat_pos / total_pred) if total_pred > 0 else None

    # Calcular métricas de avaliação
    TP = ((df['Crypto_Return_Today'] > 0) & (df['exp_value'] > 0)).sum()
    TN = ((df['Crypto_Return_Today'] < 0) & (df['exp_value'] < 0)).sum()
    FP = ((df['Crypto_Return_Today'] < 0) & (df['exp_value'] > 0)).sum()
    FN = ((df['Crypto_Return_Today'] > 0) & (df['exp_value'] < 0)).sum()
    mae = df['ae'].mean()
    acc = (TP + TN) / (TP + FP + FN + FP) if (TP + FP + FN + FP) > 0 else None
    sen = TP / (TP + FN) if (TP + FN) > 0 else None
    spe = TN / (TN + FP) if (TN + FP) > 0 else None
    pre = TP / (TP + FP) if (TP + FP) > 0 else None

    # Lower bound dos 10% maiores
    limite_90 = df['exp_value'].quantile(0.9)
    menor_valor_10_maiores = df[df['exp_value'] >= limite_90]['exp_value'].min()

    # Precisão para valores maiores que 0.008
    TP_threshold = ((df['Crypto_Return_Today'] > 0) & (df['exp_value'] > 0) & (df['exp_value'] >= 0.0002)).sum()
    FP_threshold = ((df['Crypto_Return_Today'] < 0) & (df['exp_value'] > 0) & (df['exp_value'] >= 0.0002)).sum()
    prec_maior_008 = TP_threshold / (TP_threshold + FP_threshold) if (TP_threshold + FP_threshold) > 0 else None

    # Adicionando os resultados para a criptomoeda atual
    results.append({
        "Crypto": crypto,
        "Num Subidas (Real)": num_positive,
        "Perc Subidas (Real)": perc_positive_real,
        "Num Quedas (Real)": num_negative,
        "Num Subidas (Pred)": hat_pos,
        "Perc Subidas (Pred)": perc_positive_pred,
        "Num Quedas (Pred)": hat_neg,
        "MAE": mae,
        "Acc": acc,
        "Sen": sen,
        "Spe": spe,
        "Pre": pre,
        "Lower bound dos 10% maiores": menor_valor_10_maiores,
        "Precisao para maiores que 0.008": prec_maior_008
    })

# Criando um DataFrame com os resultados
results_df = pd.DataFrame(results)

# Exibindo a tabela com os resultados

# Exibindo a tabela com os resultados
#results_df.to_csv("resultado_arquitetura_1.csv", index = True)

In [27]:
data_aux = {}
for crypto in cryptos:
    data_aux[crypto] = pd.read_csv(f"{crypto}.csv", index_col='Date')

In [28]:
data_aux['BTC-USD'].dropna()

,Crypto_Return_Today,exp_value,ae
Date,,,
2017-11-18 00:00:00+00:00,0.012002,0.009322,0.002680
2017-11-19 00:00:00+00:00,0.034233,0.007436,0.026797
2017-11-20 00:00:00+00:00,0.019899,0.006029,0.013869
2017-11-21 00:00:00+00:00,-0.016524,0.009679,0.026203
2017-11-22 00:00:00+00:00,0.021505,0.005442,0.016063
...,...,...,...
2018-04-13 00:00:00+00:00,-0.000649,0.005842,0.006492
2018-04-14 00:00:00+00:00,0.014069,0.002940,0.011129
2018-04-15 00:00:00+00:00,0.040399,0.010831,0.029568


In [38]:
data_aux['BTC-USD'][:-2400]
data_complete['BTC-USD'][-2400:]

,Crypto_Return_Today,exp_value,ae
Date,,,
2018-04-18 00:00:00+00:00,0.027192,0.009423,0.017769
2018-04-19 00:00:00+00:00,0.016415,0.003857,0.012558
2018-04-20 00:00:00+00:00,0.065273,0.005372,0.059900
2018-04-21 00:00:00+00:00,0.005274,0.005664,0.000390
2018-04-22 00:00:00+00:00,-0.013832,0.001993,0.015824
...,...,...,...
2024-11-07 00:00:00+00:00,0.003534,NaN,NaN
2024-11-08 00:00:00+00:00,0.008431,NaN,NaN
2024-11-09 00:00:00+00:00,0.002904,NaN,NaN


In [39]:
for crypto in cryptos:
    df_aux = data_complete[crypto]
    df_aux = pd.concat([data_aux[crypto][:-2400], df_aux[-2400:]])
    df_aux.index = pd.to_datetime(df_aux.index)
    data_complete[crypto] = df_aux


In [40]:
for crypto in cryptos:
    data_complete[crypto]= data_complete[crypto][~data_complete[crypto].index.duplicated(keep='first')]

In [41]:
data_complete['BTC-USD'].dropna()

,Crypto_Return_Today,exp_value,ae
Date,,,
2017-11-18 00:00:00+00:00,0.012002,0.009322,0.002680
2017-11-19 00:00:00+00:00,0.034233,0.007436,0.026797
2017-11-20 00:00:00+00:00,0.019899,0.006029,0.013869
2017-11-21 00:00:00+00:00,-0.016524,0.009679,0.026203
2017-11-22 00:00:00+00:00,0.021505,0.005442,0.016063
...,...,...,...
2018-10-30 00:00:00+00:00,-0.000437,-0.002050,0.001613
2018-10-31 00:00:00+00:00,-0.003063,-0.001979,0.001084
2018-11-01 00:00:00+00:00,0.009395,-0.001207,0.010602


In [42]:
for crypto in cryptos:
    # Define o nome do arquivo como o nome da criptomoeda seguido de .csv
    filename = f"{crypto}.csv"

    # Salva o DataFrame data_complete[crypto] no arquivo .csv
    data_complete[crypto].to_csv(filename, index=True)

    print(f"Arquivo {filename} salvo com sucesso!")

Arquivo BTC-USD.csv salvo com sucesso!
Arquivo LTC-USD.csv salvo com sucesso!


In [43]:
data_visual = data_complete.copy()

In [44]:
results = []

for crypto in cryptos:
    df = data_visual[crypto]
    df['ae'] = df['exp_value'] - df['Crypto_Return_Today']
    df['ae'] = abs(df['ae'])
    df = df.dropna()

    # Cálculos dos valores
    num_positive = (df['Crypto_Return_Today'] > 0).sum()
    num_negative = (df['Crypto_Return_Today'] < 0).sum()
    hat_pos = (df['exp_value'] > 0).sum()
    hat_neg = (df['exp_value'] < 0).sum()

    # Calcula a porcentagem de subidas e quedas reais e previstas
    total_real = num_positive + num_negative
    total_pred = hat_pos + hat_neg
    perc_positive_real = (100 * num_positive / total_real) if total_real > 0 else None
    perc_positive_pred = (100 * hat_pos / total_pred) if total_pred > 0 else None

    # Calcular métricas de avaliação
    TP = ((df['Crypto_Return_Today'] > 0) & (df['exp_value'] > 0)).sum()
    TN = ((df['Crypto_Return_Today'] < 0) & (df['exp_value'] < 0)).sum()
    FP = ((df['Crypto_Return_Today'] < 0) & (df['exp_value'] > 0)).sum()
    FN = ((df['Crypto_Return_Today'] > 0) & (df['exp_value'] < 0)).sum()
    mae = df['ae'].mean()
    acc = (TP + TN) / (TP + FP + FN + FP) if (TP + FP + FN + FP) > 0 else None
    sen = TP / (TP + FN) if (TP + FN) > 0 else None
    spe = TN / (TN + FP) if (TN + FP) > 0 else None
    pre = TP / (TP + FP) if (TP + FP) > 0 else None

    # Lower bound dos 10% maiores
    limite_90 = df['exp_value'].quantile(0.9)
    menor_valor_10_maiores = df[df['exp_value'] >= limite_90]['exp_value'].min()

    # Precisão para valores maiores que 0.008
    TP_threshold = ((df['Crypto_Return_Today'] > 0) & (df['exp_value'] > 0) & (df['exp_value'] >= 0.03)).sum()
    FP_threshold = ((df['Crypto_Return_Today'] < 0) & (df['exp_value'] > 0) & (df['exp_value'] >= 0.03)).sum()
    prec_maior_008 = TP_threshold / (TP_threshold + FP_threshold) if (TP_threshold + FP_threshold) > 0 else None

    # Adicionando os resultados para a criptomoeda atual
    results.append({
        "Crypto": crypto,
        "Num Subidas (Real)": num_positive,
        "Perc Subidas (Real)": perc_positive_real,
        "Num Quedas (Real)": num_negative,
        "Num Subidas (Pred)": hat_pos,
        "Perc Subidas (Pred)": perc_positive_pred,
        "Num Quedas (Pred)": hat_neg,
        "MAE": mae,
        "Acc": acc,
        "Sen": sen,
        "Spe": spe,
        "Pre": pre,
        "Lower bound dos 10% maiores": menor_valor_10_maiores,
        "Precisao para maiores que 0.008": prec_maior_008
    })

# Criando um DataFrame com os resultados
results_df = pd.DataFrame(results)

# Exibindo a tabela com os resultados

# Exibindo a tabela com os resultados
results_df.to_csv("resultado_arquitetura_1.csv", index = True)
results_df

,Crypto,Num Subidas (Real),Perc Subidas (Real),Num Quedas (Real),Num Subidas (Pred),Perc Subidas (Pred),Num Quedas (Pred),MAE,Acc,Sen,Spe,Pre,Lower bound dos 10% maiores,Precisao para maiores que 0.008
0,BTC-USD,189,53.846154,162,199,56.695157,152,0.032268,0.474531,0.566138,0.432099,0.537688,0.009322,NaN
1,LTC-USD,158,45.014245,193,210,59.829060,141,0.044211,0.423858,0.582278,0.388601,0.438095,0.010790,0.5


In [45]:
Pre_media = results_df['Pre'].mean()
Acc_media = results_df['Acc'].mean()
MAE_media = results_df['MAE'].mean()
Precisao_25_media = results_df['Precisao para maiores que 0.008'].mean()
print("Pre_media ", Pre_media)
print("Acc_media ", Acc_media)
print("MAE_media ", MAE_media)
print("Precisao_25_media", Precisao_25_media)

Pre_media  0.4878918401531467
Acc_media  0.44919434955975013
MAE_media  0.0382395528465254
Precisao_25_media 0.5


gerar uns graficos aqui

## 3.3 Método 2 para os ultimos X intervalos

* Aplicar bagging
* Aplicar CV e Otimização de HP
* Fazer treinamento para uma janela W maior de tempo

In [ ]:
# for day in data[crypto]:
#       day = bagging(day)

# for day in data[crypto]:
#       for sample in day:
#               sample = CV(sample) # transforma sample em um vetor de folds. Ultimos 10% do fold são validation e 90% são pure train

# for day in data[crypto]:
#       for sample in day:
#               for fold in sample:
#                       y_hat = model(fold)
#               y_hat_mean = y_hat / len(sample)
#       y_hat_mean_mean = y_hat_mean / len(day)